In [ ]:
library(data.table)
load('../../data/genomes/hg38/exonicvsnon_counts.rda') # Probabilities of each mutation type
load('../../data/genomes/hg38/gene_pd_exon.rda') # Load tables giving probability of insertion for each gene in a diploid genome for both genders
source('./src_sompop_v0_2_4.r')
xy_genes <- gene_pd_m$gene_id[gene_pd_m$chrom %in% c('X','Y')] # Getting list of genes on chroms. X and Y by symbol`

In [ ]:
geneList <- read.csv('../../data/gene_lists/drivers_lusc_filt.txt',header=F)
geneList <- geneList$V1

In [ ]:
gene_pd_m[gene_pd_m$gene_id %in% geneList]
geneList <- gene_pd_m[gene_pd_m$gene_id %in% geneList]$gene_sym

### Define parameters

In [ ]:
###################################################3
# Gender of organism
gender <- 'male'

# Symbol of driver gene disrupted at t=0
driverGene <- ''

# List of driver genes
geneList <- geneList

# Initial number of cells
N0 <- 1e3

# Average # L1 insertions / cell cycle
mu <- 1
if (gender=='male') {
    mu <- mu*(1-pd_exvsnon_m[2])# Scale by 1-probability of null (intron) insertion, to convert to probability of exonic insertion
} else if (gender=='female') {
    mu <- mu*(1-pd_exvsnon_f[2])
}

# Selective strengths of each mutation type
spd <- 0.1 # Homozygous driver
spp <- 0.005 # Homozygous passenger
sld <- spd*0.1 # Heterozygous driver
slp <- spp*0.1 # Hetero. passenger

# Number of time steps to simulate
NT <- 1e1

###################################################3
# Time resolution: number of timesteps per generation
tau <- 1
tau <- 1/tau
# Buffer size of population data object; represents the max possible number of clones in the population
maxNClones <- N0*5
# Log path
logpath <- './log.txt'

## Test run

In [ ]:
out <- sompop(N0, mu, tau, NT, sld, slp, spd, spp, gender, driverGene, geneList, maxNClones, logpath)
Pop <- out[[1]]
N <- out[[2]]
genTime <- out[[4]]

# head(Pop)
N[N==0] <- NA
genTime[genTime==0] <- NA
plot(1:NT*tau,N,type='l',xlab='Generation',ylab='Population Size')

## Batch run

In [ ]:
nrun <- 0

N0v <- c(1e2,5e2,1e3)
muv <- c(0.05,0.1,0.5,1,5)
if (gender=='male') {
    muv_adj <- muv*(1-pd_exvsnon_m[2])
} else if (gender=='female') {
    muv_adj <- muv*(1-pd_exvsnon_f[2])
}
sdv <- c(0.001,.005,0.01,0.05,0.1,0.5,1)
spv <- c(0.001,.005,0.01,0.05,0.1,0.5,1)
hef <- 0.1 # Fraction of homozygous effect assigned to heterozygous mutations

outPath <- '~/jackgl/lusc_1115_3/'
logPath <- paste0(outPath,'run_1.log')
system(paste0('rm ',logPath))

tryCatch({
    for (ii in 1:length(N0v)){
        for (jj in 1:length(muv)) {
            for (ll in 1:length(sdv)) {
                for (mm in 1:length(spv)) {
                    for (nn in 1:5) {
                        nrun <- nrun+1
                        N0 <- N0v[ii]
                        mu <- muv_adj[jj]
                        sd <- sdv[ll]
                        sp <- spv[mm]
                        line <- paste0('Run: ',toString(nrun),'\tN0: ',toString(N0v[ii]),'\tmu: ',toString(muv[jj]),'\tsd: ',toString(sdv[ll]),'\tsp: ',toString(spv[mm]))
                        write(line,file=logPath,append=TRUE)
                        out <- sompop(N0, mu, tau, NT, sd*hef, sp*hef, sd, sp, gender, driverGene, geneList, N0v[ii]*4, logPath)
                        Pop <- out[[1]]
                        N <- out[[2]]
                        mut_genes <- out[[3]]
                        gen_time <- out[[4]]
                        save(Pop,N,mut_genes,gen_time,N0,mu,sd,sp,file=paste0(outPath,'run_n0',N0v[ii],'_',nrun,".rda"))
                        rm(Pop,N)
                    }
                }
            }
        }
    }
    }, warning = function(warn.msg){
        write(toString(warn.msg), logPath, append=TRUE)
    }, error = function(err.msg){
        write(toString(err.msg), logPath, append=TRUE)
    }
)